###Install Requirements

In [1]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 15002, done.
remote: Total 15002 (delta 0), reused 0 (delta 0), pack-reused 15002
Receiving objects: 100% (15002/15002), 14.02 MiB | 23.42 MiB/s, done.
Resolving deltas: 100% (10291/10291), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 KB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 4.6 MB/s eta 0:00:00
Setup complete. Using torch 1.13.1+cu116 (Tesla T4)


In [2]:
!pip install python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=4d5883c3fbf5683e0a7173c567e09693f838d1c52b3d31caf8a99e3fba5634a4
  Stored in directory: /root/.cache/pip/wheels/9e/fc/1c/cf980e6413d3ee8e70cd8f39e2366b0f487e3e221aeb452eb0
Successfully built python-multipart


In [3]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6

###Import Library

In [4]:
# importing cv2 
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import shutil


In [5]:

import requests
from PIL import Image
from io import BytesIO

In [6]:
import gradio as gr

In [7]:
import tensorflow as tf

In [8]:
import re

#Functions

In [9]:
def createDirectory(path):
  # checking if the directory demo_folder 
  # exist or not
  if not os.path.exists(path):
    # if the demo_folder directory is not present 
    # then create it
    os.makedirs(path)

def removeDirectory(path):
  # checking if the directory demo_folder 
  # exist or not
  if os.path.exists(path):
    # if the demo_folder directory is present 
    # then remove it
    shutil.rmtree(path)


In [46]:
reObjLabel = re.compile(r"(?P<label>\S+) (?P<center_x>\S+) (?P<center_y>\S+) (?P<w>\S+) (?P<h>\S+) (?P<percent>\S+)")

def readLabels(lstCoordObj, width, height):
  lst_x0 = []
  lst_y0 = []
  lst_x1 = []
  lst_y1 = []
  lst_per = []
  for line in lstCoordObj.strip().split('\n'):
    reLabel = reObjLabel.match(line)
    center_x = int(float(reLabel.group('center_x'))*width)
    center_y = int(float(reLabel.group('center_y'))*height)
    w = int(float(reLabel.group('w'))*width)
    h = int(float(reLabel.group('h'))*height)
    x = int(center_x - w/2)
    y = int(center_y - h/2)
    lst_x0.append(x)
    lst_y0.append(y)
    lst_x1.append(x + w)
    lst_y1.append(y + h)
    lst_per.append(float(reLabel.group('percent')))
  dict_obj_detect = {'x0':lst_x0, 'y0':lst_y0, 'x1':lst_x1, 'y1':lst_y1, 'percent':lst_per }
  return dict_obj_detect

def cutImage(img, dict_obj_IO, width, height):
  size_ = len(dict_obj_IO['x0'])

  lst_img = []
  for i in range(size_):
    x0 = dict_obj_IO['x0'][i]
    y0 = dict_obj_IO['y0'][i]
    x1 = dict_obj_IO['x1'][i]
    y1 = dict_obj_IO['y1'][i]
    percent_detection = dict_obj_IO['percent'][i]
    if (percent_detection > .85):
      tmp_img = img[y0:y1, x0:x1]
      tmp_img = cv2.resize(tmp_img, (width, height))
      lst_img.append(tmp_img)
  return np.array(lst_img)

#Calibration

In [11]:
SIZE_ROW_IMG = 416

DECT_OBJ_IMG_WIDTH = 110
DECT_OBJ_IMG_HEIGHT = 20
DECT_OBJ_IMG_CHANNEL = 3

SCORE_DETECTION_CLASS_NAME = np.array(['0', '1', '2', '3', '4', '5', '6', 'L'])


In [12]:
dict_score =     {'0':0, 
                  '1':1,
                  '2':2, 
                  '3':3, 
                  '4':4, 
                  '5':5, 
                  '6':6, 
                  'L':0}

#Init

In [13]:
createDirectory('/content/row')

In [14]:
def get_clasifier_model():
  shutil.unpack_archive("/content/model.zip", '/content/model', "zip")
  #tf.config.run_functions_eagerly(True)
  model = tf.keras.models.load_model('/content/model/score_detect_classifier')
  model.compile(optimizer='adam', 
              loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)
  return model

In [16]:
model_clasifier = get_clasifier_model()


#Prediction

In [17]:
def score_localization():
  !python detect.py --weights /content/best.pt --img {SIZE_ROW_IMG} --conf 0.1 --source /content/row --save-conf --save-txt

##Predict object location

In [49]:
def get_img_from_location(filename_img, str_img_obj):
  """
  filename_img - name of row image
  str_img_obj  - string with object localization (get of function get_object_localization)
  """
  dict_obj_detect = readLabels(str_img_obj, SIZE_ROW_IMG, SIZE_ROW_IMG)
  row_img = cv2.imread(filename_img)
  lst_detect_obj = cutImage(row_img, dict_obj_detect, DECT_OBJ_IMG_WIDTH, DECT_OBJ_IMG_HEIGHT)
  return lst_detect_obj

  

In [53]:
def get_object_localization(input_img):
  removeDirectory('/content/yolov5/runs/detect/exp')
  filename = '/content/row/name.png'
  try:
    input_img = cv2.resize(input_img, (SIZE_ROW_IMG, SIZE_ROW_IMG))
    cv2.imwrite(filename, input_img)
  except Exception:
    raise Exception('Error', 'Image ')
  score_detection()
  filename_pred = '/content/yolov5/runs/detect/exp/labels/name.txt'
  f = open(filename_pred, "r")
  object_classifier = ''.join(f)
  f.close()
  
  return object_classifier

demo = gr.Interface(object_localization, gr.Image(), 'text')
demo.launch(debug=True)




Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

detect: weights=['/content/best.pt'], source=/content/row, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-72-g064365d Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/row/name.png: 416x416 8 score_evals, 8.4ms
Speed: 0.3ms pre-process, 8.4ms inference, 1.5ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp
1 labels saved to runs/detect/exp/labels
Keyboard interruption in main thread... closing server.


##Predict score

In [18]:
def score_clasifier(filename_img, filename_pred):
  
  filename_pred = '/content/yolov5/runs/detect/exp/labels/name.txt'
  f = open(filename_pred, "r")
  object_classifier = ''.join(f)
  f.close()

  dict_obj_detect = readLabels(object_classifier, SIZE_ROW_IMG, SIZE_ROW_IMG)
  row_img = cv2.imread(filename_img)
  lst_detect_obj = cutImage(row_img, dict_obj_detect, DECT_OBJ_IMG_WIDTH, DECT_OBJ_IMG_HEIGHT)
  pred_val = model_clasifier.predict(lst_detect_obj)
  pred_val = [np.argmax(pred_vec) for pred_vec in pred_val]
  return pred_val

  

In [22]:
def predict_score(input_img):
  removeDirectory('/content/yolov5/runs/detect/exp')
  filename = '/content/row/name.png'
  try:
    input_img = cv2.resize(input_img, (SIZE_ROW_IMG, SIZE_ROW_IMG))
    cv2.imwrite(filename, input_img)
  except Exception:
    raise Exception('Error', 'Image ')
  score_detection()
  filename_pred = '/content/yolov5/runs/detect/exp/labels/name.txt'
  pred_class = score_clasifier(filename, filename_pred)
  pred_score = SCORE_DETECTION_CLASS_NAME[pred_class]
  sum = 0
  for score_ in pred_score:
    sum += dict_score[score_]
  str_score = "score: {}, total score: {}".format(' '.join(pred_score), str(sum))
  return str_score

demo = gr.Interface(predict_score, gr.Image(), "text")
demo.launch(debug=True)
#demo.launch()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

detect: weights=['/content/best.pt'], source=/content/row, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-72-g064365d Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/row/name.png: 416x416 8 score_evals, 8.2ms
Speed: 0.4ms pre-process, 8.2ms inference, 1.5ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp2
1 labels saved to runs/detect/exp2/labels
1/1 [==============================] - 0s 28ms/step
['L' '4' '5' '1' '4' '1' '4']
Keyboard interruption in main thread... closing server.
